## Semantic Function Calling

In [1]:
#r "nuget:Microsoft.SemanticKernel"
#r "nuget: dotenv.net"
#r "nuget:Microsoft.SemanticKernel.Connectors.OpenAI"
#r "nuget:Microsoft.SemanticKernel.Connectors.AzureOpenAI"
#r "nuget:Aspose.Email"

Installed Packages Aspose.Email, 24.8.0 dotenv.net, 3.2.0 Microsoft.SemanticKernel, 1.19.0 Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.19.0 microsoft.semantickernel.connectors.openai, 1.19.0

In [77]:
using dotenv.net;

DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT");
var endpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at {endpoint} ");
     

Using deployment: gpt-4o at https://scribeai.openai.azure.com/ 


# Setup Plugins

In [2]:
using System.ComponentModel;
using Microsoft.SemanticKernel;
using System.Text.Json.Serialization;

public class EmailPlugin
{
    [KernelFunction("send_email")]
    [Description("Sends an email to a recipient.")]
    public async Task SendEmailAsync(
        Kernel kernel,
        List<string> recipientEmails,
        string subject,
        string body
    )
    {
        // Add logic to send an email using the recipientEmails, subject, and body
        // For now, we'll just print out a success message to the console
        Console.WriteLine("Email Plugin to send emails!");
        Console.WriteLine($"Recipients: {string.Join(", ", recipientEmails)}");
        Console.WriteLine($"Subject: {subject}");
        Console.WriteLine($"Body: {body}");
        
    }
}

public class EventModel
{
   [JsonPropertyName("person")]
   public string Person { get; set; }

   [JsonPropertyName("name")]
   public string Title { get; set; }

   [JsonPropertyName("date")]
   public DateTime Date { get; set; }

   [JsonPropertyName("notes")]
   public string Notes { get; set; }
   
}

public class CalendarPlugin
{
    [KernelFunction("get_calendar_events")]
    [Description("Gets calendar events for a given date.")]
    public async Task<List<EventModel>> GetCalendarAsync(
        Kernel kernel,
        List<string> recipientEmails,
        System.DateTime date
    )
    {
        // Add logic to get calendar events for recipientEmails on the given date
        // For now, we'll just print out a success message to the console
        Console.WriteLine("Calendar Events Plugin started");
        Console.WriteLine($"Recipients: {string.Join(", ", recipientEmails)}");
        Console.WriteLine($"date: {date}");

        // Call Graph API to get calendar events !!!!!

        return   new()
            {
                new EventModel { Person = "Child1", Title = "Soccer practice", Date=DateTime.Parse("10/16/2024 12:00:00 AM"), Notes = "bring socks" },
                new EventModel { Person = "Child2", Title = "Dentist Appointment", Date=DateTime.Parse("10/15/2024 12:00:00 AM"), Notes = "bring healthcard, no food"},
                new EventModel { Person = "Adult2", Title = "Date night", Date=DateTime.Parse("10/15/2024 09:00:00 AM"), Notes = "Marche"}
            };
    }
}

In [79]:
using Aspose.Email;
using System;

public class EmailExtractionPlugin
{
    [KernelFunction("get_event_text")]
    [Description("Extract data from Email")]
    public async Task<string> ExtractEventFromEmail(
        Kernel kernel,
        string email_path
    )
    {
        // Add logic to send an email using the recipientEmails, subject, and body
        // For now, we'll just print out a success message to the console
        Console.WriteLine("EmailAnalysis Plugin Started");

        MailMessage mail = MailMessage.Load("./data/" + "Event_Reminder_Development.eml");
        string body_without_url = mail.GetHtmlBodyText(false);// body will not contain URL
        // Console.WriteLine("Body with URL: " + body_with_url);
        // Console.WriteLine("Body without URL: " + body_without_url);  

        return body_without_url;

    }
}

# ASK Boss for Vacation

In [12]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;



// Create kernel with an email plugin
var builder = Kernel.CreateBuilder();
builder.Plugins.AddFromType<EmailPlugin>();
builder.Plugins.AddFromType<CalendarPlugin>();
builder.Plugins.AddFromType<EmailExtractionPlugin>();
Kernel kernel = builder.Build();

// Create chat completion service

AzureOpenAIChatCompletionService chatCompletionService = new (
    deploymentName: deploymentName,
    apiKey: apiKey,
    endpoint: endpoint
 );

// Enable planning
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

Console.WriteLine($" --------------------------------");
// Create chat history
var history = new ChatHistory();
history.AddUserMessage("Can you help me Elena write an email for my boss at boss@gmail.com subject: oh mine asking for vacation for days of Oct 15th and 16th in professional tone?");

// Get the response from the AI
var result = await chatCompletionService.GetChatMessageContentAsync(
    history,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel
);

Console.WriteLine($" --------------------------------");
Console.WriteLine($" Email : {result}");
Console.WriteLine($" --------------------------------");



 --------------------------------
 --------------------------------
 Email : Sure, here's a professional draft for your vacation request email:

---

**Subject:** Vacation Request for October 15th and 16th

**Body:**

Dear [Boss's Name],

I hope this message finds you well.

I am writing to formally request vacation days for October 15th and 16th. During this time, I will ensure that all my ongoing projects and responsibilities are handed over to my colleagues and that all pending tasks are completed before my leave.

I would be grateful if you could approve this request at your earliest convenience.

Thank you for considering my application. Please let me know if you need any further information.

Best regards,  
Elena

---

Would you like to send this email now?
 --------------------------------


# ASK Calendar events for vacation time

In [4]:
history.AddUserMessage("What are my events for vacation days,just list all of them, please?");

// Get the response from the AI
var vacresult = await chatCompletionService.GetChatMessageContentAsync(
    history,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel
);
Console.WriteLine($" --------------------------------");
Console.WriteLine($" Vacation events : {vacresult}");

Calendar Events Plugin started
Recipients: elena@gmail.com
date: 10/15/2023 12:00:00 AM
Calendar Events Plugin started
Recipients: elena@gmail.com
date: 10/16/2023 12:00:00 AM
 --------------------------------
 Vacation events : You have the following events scheduled for October 15th and 16th:

### October 15th
1. **Dentist Appointment for Child2**
   - Notes: bring health card, no food
   
2. **Date night for Adult2**
   - Time: 9:00 AM
   - Notes: Marche

### October 16th
1. **Soccer practice for Child1**
   - Notes: bring socks

Now let's write the email to your boss.

---

**Subject:** Vacation Request for October 15th and 16th

**Body:**

Dear [Boss's Name],

I hope this email finds you well.

I am writing to formally request vacation days on October 15th and 16th. I have ensured that my current projects are up-to-date and have delegated any urgent tasks to my colleagues during my absence.

I appreciate your understanding and consideration.

Best regards,  
Elena

---

Would you 

In [82]:
history.Clear();
history.AddUserMessage("I have received email stored at './data/Event_Reminder_Development.eml' what event it descibes and do I have other events on the date mentioned in email?");

// Get the response from the AI
var evresult = await chatCompletionService.GetChatMessageContentAsync(
    history,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel
);
Console.WriteLine($" --------------------------------");
Console.WriteLine($" events : {evresult}");

EmailAnalysis Plugin Started
Calendar Events Plugin started
Recipients: 
date: 10/8/2023 8:00:00 PM
 --------------------------------
 events : The email describes an event called "Development Centre" on **Wednesday, September 11, 2024:**

- **Time:** 6:30 PM – 7:30 PM 
  - **Arrival:** 6:25 PM
- **Location:** NSG (Windsor Bubble) 
  - Address: 808 Lytton St North Vancouver, BC, Canada
- **What to bring:**
  - Players dressed in cleats, socks, shin guards, shorts/pants, t-shirt/sweater.
  - Hair tied up.
  - Water bottle.
  - No jewelry.
- **Notes:**
  - Volunteer to bring snacks, drinks, or help with other tasks.

As for the events in your calendar, none are scheduled on the exact date of 11th September. Here are the upcoming events listed:

1. **Soccer practice** for Child1 on **October 16, 2024**.
   - Note: bring socks.
2. **Dentist Appointment** for Child2 on **October 15, 2024**.
   - Note: bring health card, no food.
3. **Date night** for Adult2 on **October 15, 2024**.
   - Not